In [38]:
import os
import time
import csv
import numpy as np
import pandas as pd
import requests
import mysql.connector
from datetime import datetime
from mysql.connector import Error

In [2]:
pip install passlib[bcrypt]


Note: you may need to restart the kernel to use updated packages.


In [ ]:


# ============================ #
# 🎮 API RAWG.IO Configuration  #
# ============================ #
API_KEY = "a596903618f14aeeb1fcbbb790180dd5"
PAGE_SIZE = 40  # Nombre de jeux par page (RAWG)
TOTAL_GAMES = 500  # Nombre total de jeux à récupérer à chaque exécution
CSV_FILENAME = "games_data.csv"  # Nom du fichier CSV

# ============================ #
# 🔍  Récupération des jeux RAWG #
# ============================ #
def fetch_games_from_rawg():
    """
    Récupère 500 nouveaux jeux depuis l'API RAWG.io et les structure dans un DataFrame.
    """
    jeux = []
    page = 1

    while len(jeux) < TOTAL_GAMES:
        url = f"https://api.rawg.io/api/games?key={API_KEY}&page_size={PAGE_SIZE}&page={page}"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"⚠️ Erreur {response.status_code} lors de l'appel à l'API.")
            break

        data = response.json()
        if "results" not in data:
            print("⚠️ Erreur : Aucun résultat trouvé dans la réponse de l'API.")
            break

        for game in data["results"]:
            genres = ", ".join([genre["name"] for genre in game.get("genres", [])])
            platforms = ", ".join([platform["platform"]["name"] for platform in game.get("platforms", [])])

            jeux.append({
                "game_id_rawg": game["id"],
                "title": game["name"],
                "release_date": game.get("released", None),
                "genres": genres if genres else None,
                "platforms": platforms if platforms else None,
                "rating": game.get("rating", None),
                "metacritic": game.get("metacritic", None),
                "last_update": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            })

        print(f"📄 Page {page} récupérée, total de jeux : {len(jeux)}")
        page += 1
        time.sleep(1)

        if len(jeux) >= TOTAL_GAMES:
            break

    return pd.DataFrame(jeux[:TOTAL_GAMES])

# ============================ #
# 💾  Enregistrement MySQL      #
# ============================ #
def save_games_to_mysql(df_games):
    """
    Enregistre les jeux dans la base de données MySQL en évitant les doublons.
    """
    conn = None
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1", port=3307,
            user="root",
            password="root",
            database="games_db"
        )
        cursor = conn.cursor()
        create_table_query = """
        CREATE TABLE IF NOT EXISTS games (
            game_id_rawg INT PRIMARY KEY,
            title VARCHAR(255),
            release_date DATE,
            genres TEXT,
            platforms TEXT,
            rating FLOAT,
            metacritic INT,
            last_update DATETIME DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
        )
        """
        cursor.execute(create_table_query)
        conn.commit()
        
        # Remplacement des NaN par None pour éviter les erreurs MySQL
        df_games = df_games.replace({np.nan: None})
        
        insert_query = """
        INSERT INTO games (game_id_rawg, title, release_date, genres, platforms, rating, metacritic, last_update)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
        title = VALUES(title),
        release_date = VALUES(release_date),
        genres = VALUES(genres),
        platforms = VALUES(platforms),
        rating = VALUES(rating),
        metacritic = VALUES(metacritic),
        last_update = VALUES(last_update);
        """
        cursor.executemany(insert_query, df_games.values.tolist())
        conn.commit()
        print(f"✅ {len(df_games)} jeux enregistrés/actualisés dans MySQL.")
    except Error as e:
        print(f"Erreur MySQL : {e}")
    finally:
        if conn and conn.is_connected():
            cursor.close()
            conn.close()
            print("Connexion MySQL fermée.")

# ============================ #
# 🚀  Exécution du programme   #
# ============================ #
if __name__ == "__main__":
    print("\n📌 Récupération de 500 nouveaux jeux via l'API RAWG.io...")
    df_games = fetch_games_from_rawg()

    print("\n📌 Ajout des jeux dans MySQL...")
    save_games_to_mysql(df_games)

    print("\n🎉 Programme terminé avec succès !")


📌 Récupération de 500 nouveaux jeux via l'API RAWG.io...
📄 Page 1 récupérée, total de jeux : 40
📄 Page 2 récupérée, total de jeux : 80
📄 Page 3 récupérée, total de jeux : 120
📄 Page 4 récupérée, total de jeux : 160
📄 Page 5 récupérée, total de jeux : 200
📄 Page 6 récupérée, total de jeux : 240
📄 Page 7 récupérée, total de jeux : 280
📄 Page 8 récupérée, total de jeux : 320
📄 Page 9 récupérée, total de jeux : 360
📄 Page 10 récupérée, total de jeux : 400
📄 Page 11 récupérée, total de jeux : 440
📄 Page 12 récupérée, total de jeux : 480
📄 Page 13 récupérée, total de jeux : 520

📌 Ajout des jeux dans MySQL...
✅ 500 jeux enregistrés/actualisés dans MySQL.
Connexion MySQL fermée.

🎉 Programme terminé avec succès !


In [23]:
import requests
import pandas as pd
import time
from datetime import datetime
import mysql.connector
from mysql.connector import Error
import numpy as np

# ============================ #
# 🎮 API RAWG.IO Configuration  #
# ============================ #
API_KEY = "a596903618f14aeeb1fcbbb790180dd5"
PAGE_SIZE = 40  # Nombre de jeux par page (RAWG)
TOTAL_GAMES = 500  # Nombre total de jeux à récupérer à chaque exécution
CSV_FILENAME = "games_data.csv"  # Nom du fichier CSV
RAWG_MAX_PAGES = 20000  # Nombre total de pages RAWG (limite de pagination)

# ============================ #
# 📌 Connexion MySQL            #
# ============================ #
def connect_to_db():
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1", port=3307,
            user="root",
            password="root",
            database="games_db"
        )
        return conn
    except Error as e:
        print(f"❌ Erreur MySQL : {e}")
        return None

# ============================ #
# 📌 Récupérer la dernière page #
# ============================ #
def get_last_page():
    conn = connect_to_db()
    if not conn:
        return 1  # Par défaut, commencer à la page 1
    cursor = conn.cursor()
    cursor.execute("CREATE TABLE IF NOT EXISTS api_state (id INT PRIMARY KEY, last_page INT DEFAULT 1)")
    conn.commit()
    cursor.execute("SELECT last_page FROM api_state WHERE id = 1")
    row = cursor.fetchone()
    conn.close()
    return row[0] if row else 1  # Retourne la dernière page enregistrée

# ============================ #
# 📌 Mettre à jour la page RAWG #
# ============================ #
def update_last_page(page):
    conn = connect_to_db()
    if not conn:
        return
    cursor = conn.cursor()
    cursor.execute("INSERT INTO api_state (id, last_page) VALUES (1, %s) ON DUPLICATE KEY UPDATE last_page = %s", (page, page))
    conn.commit()
    conn.close()

# ============================ #
# 🔍  Récupération des jeux RAWG #
# ============================ #
def fetch_games_from_rawg():
    """Récupère 500 nouveaux jeux en évitant les doublons et en continuant la pagination."""
    jeux = []
    page = get_last_page()  # Récupérer la dernière page connue
    existing_ids = get_existing_game_ids()
    print(f"📌 Nombre de jeux déjà en base : {len(existing_ids)}")
    print(f"📌 Dernière page RAWG utilisée : {page}")
    
    while len(jeux) < TOTAL_GAMES:
        url = f"https://api.rawg.io/api/games?key={API_KEY}&page_size={PAGE_SIZE}&page={page}"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"⚠️ Erreur {response.status_code} lors de l'appel à l'API.")
            break

        data = response.json()
        if "results" not in data:
            print("⚠️ Erreur : Aucun résultat trouvé dans la réponse de l'API.")
            break

        for game in data["results"]:
            if game["id"] in existing_ids:
                print(f"⛔ Jeu ignoré car déjà en base : {game['name']} (ID: {game['id']})")
            else:
                print(f"✅ Nouveau jeu trouvé : {game['name']} (ID: {game['id']})")
                genres = ", ".join([genre["name"] for genre in game.get("genres", [])])
                platforms = ", ".join([platform["platform"]["name"] for platform in game.get("platforms", [])])

                jeux.append({
                    "game_id_rawg": game["id"],
                    "title": game["name"],
                    "release_date": game.get("released", None),
                    "genres": genres if genres else None,
                    "platforms": platforms if platforms else None,
                    "rating": game.get("rating", None),
                    "metacritic": game.get("metacritic", None),
                    "last_update": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
        
        print(f"📄 Page {page} récupérée, total de nouveaux jeux : {len(jeux)}")
        page = page + 1 if page < RAWG_MAX_PAGES else 1  # Revenir à 1 si on atteint la fin
        time.sleep(1)

        if len(jeux) >= TOTAL_GAMES:
            break

    update_last_page(page)  # Mettre à jour la dernière page utilisée
    return pd.DataFrame(jeux)

# ============================ #
# 📌 Récupérer les jeux existants #
# ============================ #
def get_existing_game_ids():
    conn = connect_to_db()
    if not conn:
        return set()
    cursor = conn.cursor()
    cursor.execute("SELECT game_id_rawg FROM games")
    existing_ids = {row[0] for row in cursor.fetchall()}
    conn.close()
    return existing_ids

# ============================ #
# 💾  Enregistrement MySQL      #
# ============================ #
def save_games_to_mysql(df_games):
    """Enregistre les nouveaux jeux en évitant les doublons."""
    conn = connect_to_db()
    if not conn:
        return

    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS games (
            game_id_rawg INT PRIMARY KEY,
            title VARCHAR(255),
            release_date DATE,
            genres TEXT,
            platforms TEXT,
            rating FLOAT,
            metacritic INT,
            last_update DATETIME DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
        )
    """)
    conn.commit()
    
    df_games = df_games.replace({np.nan: None})
    
    insert_query = """
        INSERT INTO games (game_id_rawg, title, release_date, genres, platforms, rating, metacritic, last_update)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE last_update = VALUES(last_update);
    """
    cursor.executemany(insert_query, df_games.values.tolist())
    conn.commit()
    print(f"✅ {len(df_games)} nouveaux jeux enregistrés.")

    cursor.close()
    conn.close()

# ============================ #
# 🚀  Exécution du programme   #
# ============================ #
if __name__ == "__main__":
    print("📌 Récupération de 500 nouveaux jeux...")
    df_games = fetch_games_from_rawg()

    if not df_games.empty:
        print("📌 Ajout des nouveaux jeux dans MySQL...")
        save_games_to_mysql(df_games)
    else:
        print("✅ Aucun nouveau jeu à ajouter !")

    print("🎉 Programme terminé avec succès !")

📌 Récupération de 500 nouveaux jeux...
📌 Nombre de jeux déjà en base : 2040
📌 Dernière page RAWG utilisée : 52
✅ Nouveau jeu trouvé : Yes, Your Grace (ID: 379556)
✅ Nouveau jeu trouvé : PGA TOUR 2K21 (ID: 442864)
✅ Nouveau jeu trouvé : Google Earth VR (ID: 9789)
✅ Nouveau jeu trouvé : The Room Two (ID: 5916)
✅ Nouveau jeu trouvé : Lichdom: Battlemage (ID: 1039)
✅ Nouveau jeu trouvé : Galactic Civilizations III (ID: 9668)
✅ Nouveau jeu trouvé : Ascendant (ID: 3181)
✅ Nouveau jeu trouvé : Ultimate Marvel vs. Capcom 3 (ID: 713)
✅ Nouveau jeu trouvé : WARSAW (ID: 297320)
✅ Nouveau jeu trouvé : Insurgency: Sandstorm (ID: 59306)
✅ Nouveau jeu trouvé : Age of Empires III: Definitive Edition (ID: 499264)
✅ Nouveau jeu trouvé : WWE 2K Battlegrounds (ID: 437042)
✅ Nouveau jeu trouvé : Guild Wars 2 (ID: 29746)
✅ Nouveau jeu trouvé : Rise of Nations: Extended Edition (ID: 9850)
✅ Nouveau jeu trouvé : Odysseus Kosmos - Episode 1 (ID: 79236)
✅ Nouveau jeu trouvé : The Cycle: Frontier (ID: 60841)
✅ N

In [24]:
import requests

API_KEY = "a596903618f14aeeb1fcbbb790180dd5"

url = f"https://api.rawg.io/api/games?key={API_KEY}&page_size=40&page=1"
response = requests.get(url)
data = response.json()

print(data)  # Vérifier si "results" contient bien des jeux


{'count': 882916, 'next': 'https://api.rawg.io/api/games?key=a596903618f14aeeb1fcbbb790180dd5&page=2&page_size=40', 'previous': None, 'results': [{'id': 3498, 'slug': 'grand-theft-auto-v', 'name': 'Grand Theft Auto V', 'released': '2013-09-17', 'tba': False, 'background_image': 'https://media.rawg.io/media/games/20a/20aa03a10cda45239fe22d035c0ebe64.jpg', 'rating': 4.47, 'rating_top': 5, 'ratings': [{'id': 5, 'title': 'exceptional', 'count': 4226, 'percent': 59.01}, {'id': 4, 'title': 'recommended', 'count': 2343, 'percent': 32.72}, {'id': 3, 'title': 'meh', 'count': 456, 'percent': 6.37}, {'id': 1, 'title': 'skip', 'count': 136, 'percent': 1.9}], 'ratings_count': 7048, 'reviews_text_count': 65, 'added': 21891, 'added_by_status': {'yet': 546, 'owned': 12660, 'beaten': 6173, 'toplay': 622, 'dropped': 1143, 'playing': 747}, 'metacritic': 92, 'playtime': 74, 'suggestions_count': 439, 'updated': '2025-03-13T20:36:32', 'user_game': None, 'reviews_count': 7161, 'saturated_color': '0f0f0f', 'd

In [25]:
from requests import post
response = post('https://api.igdb.com/v4/platforms', **{'headers': {'Client-ID': 'Client ID', 'Authorization': 'Bearer access_token'},'data': 'fields abbreviation,alternative_name,category,checksum,created_at,generation,name,platform_family,platform_logo,platform_type,slug,summary,updated_at,url,versions,websites;'})
print ("response: %s" % str(response.json()))

response: {'message': 'Authorization Failure. Have you tried:', 'Tip 1': 'Ensure you are sending Authorization and Client-ID as headers.', 'Tip 2': "Ensure Authorization value starts with 'Bearer ', including the space", 'Tip 3': 'Ensure Authorization value ends with the App Access Token you generated, NOT your Client Secret.', 'Docs': 'https://api-docs.igdb.com/#authentication', 'Discord': 'https://discord.gg/igdb'}


In [29]:
# Remplacer les NaN par None (équivalent à NULL en SQL)
df_games = df_games.where(pd.notna(df_games), None)

In [32]:
import mysql.connector
from mysql.connector import Error
import requests
import pandas as pd

# ============================ #
# 🎮 Configuration API RAWG.IO  #
# ============================ #
API_KEY = "a596903618f14aeeb1fcbbb790180dd5"
CSV_FILENAME = "platforms_list.csv"  # Nom du fichier CSV

# ============================ #
# 📌 Connexion MySQL            #
# ============================ #
def connect_to_db():
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1", port=3307,
            user="root",
            password="root",
            database="games_db"
        )
        return conn
    except Error as e:
        print(f"❌ Erreur MySQL : {e}")
        return None

# ============================ #
# 🔍  Récupération des plateformes RAWG #
# ============================ #
def fetch_platforms_from_rawg():
    """
    Récupère la liste des plateformes depuis l'API RAWG.io.
    """
    url = f"https://api.rawg.io/api/platforms?key={API_KEY}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"⚠️ Erreur {response.status_code} lors de la requête API.")
        return pd.DataFrame()  # Retourne un DataFrame vide

    data = response.json()
    if "results" not in data:
        print("⚠️ Erreur : Aucun résultat trouvé.")
        return pd.DataFrame()

    # Extraction des plateformes
    platforms_list = [{"platform_id": p["id"], "platform_name": p["name"]} for p in data["results"]]

    return pd.DataFrame(platforms_list)

# ============================ #
# 💾  Enregistrement MySQL      #
# ============================ #
def save_platforms_to_mysql(df_platforms):
    """
    Enregistre les plateformes dans la base de données MySQL en évitant les doublons.
    """
    conn = connect_to_db()
    if not conn:
        return
    
    try:
        cursor = conn.cursor()
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS platforms (
                platform_id INT PRIMARY KEY,
                platform_name VARCHAR(255) NOT NULL
            )
        """)
        conn.commit()
        
        insert_query = """
            INSERT INTO platforms (platform_id, platform_name)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE platform_name = VALUES(platform_name);
        """
        cursor.executemany(insert_query, df_platforms.values.tolist())
        conn.commit()
        print(f"✅ {len(df_platforms)} plateformes enregistrées/actualisées dans MySQL.")
    except Error as e:
        print(f"❌ Erreur MySQL lors de l'insertion : {e}")
    finally:
        cursor.close()
        conn.close()

# ============================ #
# 💾  Sauvegarde en CSV         #
# ============================ #
def save_platforms_to_csv(df_platforms):
    """
    Enregistre les plateformes dans un fichier CSV.
    """
    df_platforms.to_csv(CSV_FILENAME, index=False, encoding="utf-8")
    print(f"✅ Données enregistrées dans {CSV_FILENAME} ({len(df_platforms)} plateformes)")

# ============================ #
# 🚀  Exécution du programme   #
# ============================ #
if __name__ == "__main__":
    print("\n📌 Récupération des plateformes via l'API RAWG.io...")
    df_platforms = fetch_platforms_from_rawg()

    if not df_platforms.empty:
        print("\n📌 Sauvegarde des plateformes en CSV...")
        save_platforms_to_csv(df_platforms)

        print("\n📌 Enregistrement des plateformes dans MySQL...")
        save_platforms_to_mysql(df_platforms)

        print("\n🎉 Programme terminé avec succès !")

        # 📌 Affichage des 10 premières plateformes
        print("\n🔹 Liste des premières plateformes récupérées :")
        print(df_platforms.head(10))
    else:
        print("\n❌ Aucune plateforme trouvée.")



📌 Récupération des plateformes via l'API RAWG.io...

📌 Sauvegarde des plateformes en CSV...
✅ Données enregistrées dans platforms_list.csv (50 plateformes)

📌 Enregistrement des plateformes dans MySQL...
✅ 50 plateformes enregistrées/actualisées dans MySQL.

🎉 Programme terminé avec succès !

🔹 Liste des premières plateformes récupérées :
   platform_id    platform_name
0            4               PC
1          187    PlayStation 5
2            1         Xbox One
3           18    PlayStation 4
4          186  Xbox Series S/X
5            7  Nintendo Switch
6            3              iOS
7           21          Android
8            8     Nintendo 3DS
9            9      Nintendo DS


In [28]:
import pandas as pd 
import os
import mysql.connector
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from datetime import datetime
from mysql.connector import Error

# ============================ #
# 📌 Connexion MySQL            #
# ============================ #
def connect_to_db():
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1", port=3307,
            user="root",
            password="root",
            database="games_db"
        )
        return conn
    except Error as e:
        print(f"❌ Erreur MySQL : {e}")
        return None

# ============================ #
# 💾  Récupérer les jeux depuis la BDD  #
# ============================ #
def fetch_games_from_db():
    conn = connect_to_db()
    if not conn:
        return pd.DataFrame()
    cursor = conn.cursor()
    cursor.execute("SELECT title FROM games")
    games = cursor.fetchall()
    conn.close()
    return pd.DataFrame(games, columns=["title"])

# ============================ #
# 💾  Enregistrement MySQL      #
# ============================ #
def save_prices_to_mysql(df_prices):
    """Enregistre les prix dans la base de données MySQL en évitant les doublons."""
    conn = connect_to_db()
    if not conn:
        return

    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS best_price_pc (
            title VARCHAR(255) PRIMARY KEY,
            best_price_PC VARCHAR(50),
            best_shop_PC VARCHAR(255),
            site_url_PC TEXT,
            last_update DATETIME DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP
        )
    """)
    conn.commit()
    
    insert_query = """
        INSERT INTO best_price_pc (title, best_price_PC, best_shop_PC, site_url_PC, last_update)
        VALUES (%s, %s, %s, %s, %s)
        ON DUPLICATE KEY UPDATE 
        best_price_PC = VALUES(best_price_PC),
        best_shop_PC = VALUES(best_shop_PC),
        site_url_PC = VALUES(site_url_PC),
        last_update = VALUES(last_update);
    """
    cursor.executemany(insert_query, df_prices.values.tolist())
    conn.commit()
    print(f"✅ {len(df_prices)} prix enregistrés/actualisés dans MySQL.")

    cursor.close()
    conn.close()

# ============================ #
# 🔍 Scraping des prix DLCompare #
# ============================ #
def scrape_best_prices():
    """Scrape les prix des jeux en base de données sur DLCompare."""
    games_data = fetch_games_from_db()
    
    if games_data.empty:
        print("❌ Aucun jeu trouvé dans la base de données.")
        return pd.DataFrame()
    
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)

    updated_prices = []
    
    for title in games_data["title"]:
        print(f"🔍 Recherche du prix pour {title}...")
        search_url = f"https://www.dlcompare.fr/search?q={title.replace(' ', '+')}#all"
        driver.get(search_url)

        try:
            game_element = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "name.clickable"))
            )
            game_element.click()
            url_pc = driver.current_url + "#pc"
            driver.get(url_pc)

            best_price_pc = None
            best_shop_pc = None

            try:
                price_element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CLASS_NAME, "lowPrice"))
                )
                best_price_pc = price_element.text

                shop_element = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "a.shop > span"))
                )
                best_shop_pc = shop_element.text
            except:
                print(f"⚠️ Aucun prix trouvé pour {title} sur PC")

            last_update = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            updated_prices.append([
                title, best_price_pc, best_shop_pc, url_pc, last_update
            ])

        except Exception as e:
            print(f"❌ Aucun résultat fiable pour {title}: {e}")
            updated_prices.append([title, None, None, None, datetime.now().strftime("%Y-%m-%d %H:%M:%S")])

    driver.quit()
    return pd.DataFrame(updated_prices, columns=["title", "best_price_PC", "best_shop_PC", "site_url_PC", "last_update"])

# ============================ #
# 🚀  Exécution du programme   #
# ============================ #
if __name__ == "__main__":
    print("📌 Scraping des prix des jeux...")
    df_prices = scrape_best_prices()

    if not df_prices.empty:
        print("📌 Enregistrement des prix dans MySQL...")
        save_prices_to_mysql(df_prices)
    else:
        print("✅ Aucun nouveau prix à enregistrer !")

    print("🎉 Programme terminé avec succès !")

📌 Scraping des prix des jeux...
🔍 Recherche du prix pour DiRT 4...
🔍 Recherche du prix pour Gran Turismo Sport...
🔍 Recherche du prix pour Middle-earth: Shadow of War...
🔍 Recherche du prix pour Nex Machina...
🔍 Recherche du prix pour Pyre...
🔍 Recherche du prix pour Red Dead Redemption 2...
🔍 Recherche du prix pour South Park: The Fractured But Whole...
🔍 Recherche du prix pour WipEout Omega Collection...
🔍 Recherche du prix pour Destiny 2...
🔍 Recherche du prix pour Final Fantasy XII: The Zodiac Age...
🔍 Recherche du prix pour Crash Bandicoot N. Sane Trilogy...
🔍 Recherche du prix pour TEKKEN 7...
🔍 Recherche du prix pour Injustice 2...
🔍 Recherche du prix pour Prey...
🔍 Recherche du prix pour Little Nightmares...
🔍 Recherche du prix pour What Remains of Edith Finch...
🔍 Recherche du prix pour Full Throttle Remastered...
🔍 Recherche du prix pour Yooka-Laylee...
🔍 Recherche du prix pour Persona 5...
🔍 Recherche du prix pour APB Reloaded...
🔍 Recherche du prix pour Punch Club...
🔍 Rech

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF787D7FE45+26629]
	(No symbol) [0x00007FF787CE6010]
	(No symbol) [0x00007FF787B7914C]
	(No symbol) [0x00007FF787BBF75F]
	(No symbol) [0x00007FF787BF7972]
	(No symbol) [0x00007FF787BF22CE]
	(No symbol) [0x00007FF787BF1379]
	(No symbol) [0x00007FF787B455E5]
	GetHandleVerifier [0x00007FF7880D72BD+3529853]
	GetHandleVerifier [0x00007FF7880EDA22+3621858]
	GetHandleVerifier [0x00007FF7880E24D3+3575443]
	GetHandleVerifier [0x00007FF787E4B77A+860474]
	(No symbol) [0x00007FF787CF088F]
	(No symbol) [0x00007FF787B441FF]
	GetHandleVerifier [0x00007FF78815ECD8+4085400]
	BaseThreadInitThunk [0x00007FFB432E259D+29]
	RtlUserThreadStart [0x00007FFB451EAF38+40]


In [ ]:
import mysql.connector  # 📌 Importer MySQL Connector
from mysql.connector import Error
import pandas as pd  # 📌 Pour gérer les erreurs MySQL

def connect_to_db():
    """
    Établit une connexion à la base de données MySQL.
    """
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1",
            port=3307,  # Vérifie que c'est bien le bon port MySQL
            user="root",
            password="root",
            database="games_db"
        )
        return conn
    except Error as e:
        print(f"❌ Erreur de connexion MySQL : {e}")
        return None

def create_game_platforms_table():
    """
    Crée la table game_platforms si elle n'existe pas déjà.
    """
    conn = connect_to_db()
    if not conn:
        print("❌ Connexion MySQL échouée.")
        return

    try:
        cursor = conn.cursor()
        create_table_query = """
            CREATE TABLE IF NOT EXISTS game_platforms (
                game_id_rawg INT NOT NULL,
                platform_id INT NOT NULL,
                PRIMARY KEY (game_id_rawg, platform_id),
                FOREIGN KEY (game_id_rawg) REFERENCES games(game_id_rawg) ON DELETE CASCADE ON UPDATE CASCADE,
                FOREIGN KEY (platform_id) REFERENCES platforms(platform_id) ON DELETE CASCADE ON UPDATE CASCADE
            );
        """
        cursor.execute(create_table_query)
        conn.commit()
        print("✅ Table game_platforms créée avec succès ou déjà existante.")
    except mysql.connector.Error as err:
        print(f"❌ Erreur MySQL lors de la création de la table : {err}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

def save_game_platforms_to_mysql(df_games):
    """
    Insère les relations entre jeux et plateformes dans la table game_platforms.
    """
    conn = connect_to_db()
    if not conn:
        print("❌ Connexion MySQL échouée.")
        return

    try:
        cursor = conn.cursor()

        insert_query = """
            INSERT INTO game_platforms (game_id_rawg, platform_id)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE platform_id = VALUES(platform_id);
        """

        relations = []
        for _, row in df_games.iterrows():
            game_id = row.get("game_id_rawg")
            platforms = row.get("platforms")

            if not game_id or pd.isna(platforms) or platforms.strip() == "":
                continue

            platform_list = [p.strip() for p in platforms.split(",")]  # Séparer et nettoyer

            for platform in platform_list:
                cursor.execute("SELECT platform_id FROM platforms WHERE platform_name = %s", (platform,))
                platform_id = cursor.fetchone()
                
                if platform_id:
                    relations.append((game_id, platform_id[0]))

        if relations:
            cursor.executemany(insert_query, relations)
            conn.commit()
            print(f"✅ {len(relations)} relations jeu-plateforme enregistrées dans MySQL.")
    except mysql.connector.Error as err:
        print(f"❌ Erreur MySQL lors de l'insertion des relations : {err}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

# 🚀 Exécution des fonctions pour garantir la création et insertion
def main():
    create_game_platforms_table()
    # Ici, df_games doit être défini avec les jeux et plateformes
    # save_game_platforms_to_mysql(df_games)  # Décommente après avoir défini df_games

if __name__ == "__main__":
    main()


✅ Table game_platforms créée avec succès ou déjà existante.


In [ ]:
def insert_game_platforms():
    """
    Insère les relations entre jeux et plateformes dans la table game_platforms.
    """
    conn = connect_to_db()
    if not conn:
        print("❌ Connexion MySQL échouée.")
        return

    cursor = conn.cursor()

    # Récupérer toutes les plateformes existantes
    cursor.execute("SELECT platform_id, platform_name FROM platforms")
    platform_mapping = {name.strip(): pid for pid, name in cursor.fetchall()}  # Création d'un dictionnaire {nom: id}

    # Récupérer les jeux avec leurs plateformes
    cursor.execute("SELECT game_id_rawg, platforms FROM games")
    games = cursor.fetchall()

    relations = []
    
    for game_id, platforms in games:
        if platforms:  # Vérifier que la colonne platforms n'est pas vide
            platform_list = [p.strip() for p in platforms.split(",") if p.strip()]  # Nettoyage

            for platform in platform_list:
                platform_id = platform_mapping.get(platform)  # Récupérer l'ID de la plateforme
                if platform_id:
                    relations.append((game_id, platform_id))
                else:
                    print(f"⚠️ Plateforme inconnue : {platform}")

    # Insérer les relations dans game_platforms
    if relations:
        insert_query = """
            INSERT INTO game_platforms (game_id_rawg, platform_id)
            VALUES (%s, %s)
            ON DUPLICATE KEY UPDATE platform_id = VALUES(platform_id);
        """
        cursor.executemany(insert_query, relations)
        conn.commit()
        print(f"✅ {len(relations)} relations jeu-plateforme enregistrées dans MySQL.")
    else:
        print("⚠️ Aucune relation à insérer.")

    cursor.close()
    conn.close()

# Exécuter l'insertion
insert_game_platforms()

✅ 8271 relations jeu-plateforme enregistrées dans MySQL.
